In [5]:
import numpy as np
import pandas as pd
from one.api import ONE

from psyfun import io

In [6]:
mode = 'auto'  # sets mode for all queries (one.seach* methods)
one = ONE(mode=mode)

In [ ]:
# This function queries the database for insertions, and returns a dataframe
# with the pids and other info about the recording

# df_recordings = io.query_recordings(one, qc='50', aligned=True, save=True)

In [7]:
# Or load from a previously saved file

df_recordings = io.load_recordings()

In [8]:
# This function loads some additional metadata derived from Guido's
# Google sheet (incl. LSD administration time)

df_meta = io.load_metadata()

In [ ]:
# This function fetches the protocol timings for a single eid

# io.fetch_protocol_timings(one, eid)

In [10]:
# Loop over eids, download the protocol timings for each, and collect
# them in a dataframe, and save as a csv

eids = df_recordings['eid'].unique()
df_timings = pd.DataFrame()
for eid in eids:
    df_timings = pd.concat([df_timings, io.fetch_protocol_timings(one, eid)])

# Helper function for when timings can't be directly recovered
def _fill_missing_timings(row):
    for protocol in [0, 1]:
        row[f'rfm_stop_0{protocol}'] = row[f'replay_start_0{protocol}']
        row[f'rfm_start_0{protocol}'] = row[f'rfm_stop_0{protocol}'] - 60 * 5
        row[f'spontaneous_stop_0{protocol}'] = row[f'rfm_start_0{protocol}']
    return row

df_timings = df_timings.apply(_fill_missing_timings, axis='columns')
df_timings.to_csv('metadata/timings.csv')

/home/openlab/Desktop/Jaime/Code/psychedelics/psyfun/io.py:103: UserWarning: No ALF data found for d77dd225-f7a9-4d01-996b-82c3ae73d56c, task 00 Reverting to raw task data
  warnings.warn(
/home/openlab/Desktop/Jaime/Code/psychedelics/psyfun/io.py:103: UserWarning: No ALF data found for d77dd225-f7a9-4d01-996b-82c3ae73d56c, task 01 Reverting to raw task data
  warnings.warn(
/home/openlab/Desktop/Jaime/Code/psychedelics/psyfun/io.py:103: UserWarning: No ALF data found for 86f743cd-ee0d-4ca5-a107-dca9a0cf3f7a, task 00 Reverting to raw task data
  warnings.warn(
/home/openlab/Desktop/Jaime/Code/psychedelics/psyfun/io.py:103: UserWarning: No ALF data found for 86f743cd-ee0d-4ca5-a107-dca9a0cf3f7a, task 01 Reverting to raw task data
  warnings.warn(
/home/openlab/Desktop/Jaime/Code/psychedelics/psyfun/io.py:103: UserWarning: No ALF data found for 4b398979-c54c-498f-a3d4-b201c2285be5, task 00 Reverting to raw task data
  warnings.warn(
/home/openlab/Desktop/Jaime/Code/psychedelics/psyfun/io

In [6]:
# Or load from a previously saved file

df_timings = io.load_timings()

In [ ]:
# This function fetches the spike times and cluster info for a single pid

# io.fetch_spikes(one, pid)

In [ ]:
# Loop over pids, download the protocol timings for each, and collect
# them in a dataframe, and save as a csv

# pids = df_recordings.index
# df_spikes = pd.DataFrame()
# for pid in tqdm(pids):
#     df_spikes = pd.concat([df_spikes, io.fetch_spikes(one, pid)])
# df_spikes.to_pickle('data/spike_times.pkl')

In [7]:
# Or load from a previously saved file

df_spikes = io.load_spikes()

### Old code for dowloading data to this repo in ALF directory structure

In [19]:
data_folder = Path ('/home/openlab/Desktop/Jaime/Code/psychedelics/Data/')
data_list = []
for eid_path in data_folder.joinpath('eids').iterdir():
    eid = eid_path.name
    for pid_path in eid_path.iterdir():
        if not any(os.scandir(pid_path)):
            print(f'No files found in {pid_path}')
        else:
            pid = pid_path.name    
            channels_file = pid_path.joinpath('channels.pkl')
            channels = pd.read_pickle(channels_file)
            spikes_file = pid_path.joinpath('spikes.pkl')
            spikes = pd.read_pickle(spikes_file)
            clusters_file = pid_path.joinpath('clusters.pkl')
            clusters = pd.read_pickle(clusters_file)
            data_list.append({'eid': eid, 
                              'pid': pid,
                              'spikes': spikes,
                              'channels': channels,
                              'clusters': clusters})
df = pd.DataFrame (data_list)

No files found in /home/openlab/Desktop/Jaime/Code/psychedelics/Data/eids/a6b9f254-7682-47a4-9abd-753013c87c70/9523391b-d87d-47e4-9f60-ee1abc431202


In [24]:
df_all = df.merge (df_timings, on='eid', how = 'left')
df_all.to_pickle('/home/openlab/Desktop/Jaime/Code/psychedelics/Data/df_all.pkl')
df_all

In [25]:
df_all = pd.read_pickle('/home/openlab/Desktop/Jaime/Code/psychedelics/Data/df_all.pkl')
df_all

,eid,pid,spikes,channels,clusters,spontaneous_start_00,spontaneous_stop_00,rfm_start_00,rfm_stop_00,replay_start_00,replay_stop_00,spontaneous_start_01,spontaneous_stop_01,rfm_start_01,rfm_stop_01,replay_start_01,replay_stop_01,admin_time
0,aff40fef-7ef3-4e7a-9451-f248e6d385db,cd02add9-e592-4c09-a2a2-8ed9576d76c8,{},{},None,92.489430,392.555194,404.821861,705.887391,2968.815543,1042.197770,2989.578826,3289.627991,3301.894658,3602.959888,4165.659292,3938.842943,1094
1,e13d0e58-55c8-4787-a716-03549b3d0240,9d0062d1-edf7-42c2-94ab-4b14f67742e7,"{'times': [0.014194586436275303, 0.01476124836...","{'x': [-0.004342, -0.004342, -0.004339, -0.004...","{'channels': [1, 0, 1, 1, 1, 3, 1, 1, 1, 1, 1,...",50.254854,350.303653,362.570320,663.635550,2911.388387,999.701986,2932.157868,3232.223232,3244.489899,3545.538331,3917.679793,3881.635231,1065
2,e13d0e58-55c8-4787-a716-03549b3d0240,820f939d-971c-4521-8424-1ed0922c5a20,"{'depths': [1945.74169921875, 983.713928222656...","{'localCoordinates': [[43.0, 20.0], [11.0, 20....",{'uuids': ['a90c6ca0-943c-4cdd-834f-d7c6362474...,50.254854,350.303653,362.570320,663.635550,2911.388387,999.701986,2932.157868,3232.223232,3244.489899,3545.538331,3917.679793,3881.635231,1065
3,53d8cb2f-3338-4bcd-81c4-6207b660eba3,1286797b-8c9a-4541-9907-b77244d2a072,"{'depths': [578.089111328125, 2171.47509765625...","{'x': [-9.6e-05, -9.6e-05, -0.000102, -0.00010...",{'uuids': ['2286007a-ac57-461e-8522-5e632dfd4b...,85.650914,385.715944,397.982611,699.047941,3006.530638,1042.013455,3027.283022,3327.331420,3339.598087,3640.663117,4009.951331,3983.424584,1146
4,53d8cb2f-3338-4bcd-81c4-6207b660eba3,101a25f8-d936-4c88-ac1f-6e39247b755f,"{'clusters': [155, 1559, 929, 268, 1060, 1296,...","{'x': [-0.001628, -0.001628, -0.00162, -0.0016...","{'depths': [20.0, 20.0, 60.0, 40.0, 40.0, 60.0...",85.650914,385.715944,397.982611,699.047941,3006.530638,1042.013455,3027.283022,3327.331420,3339.598087,3640.663117,4009.951331,3983.424584,1146
5,4b398979-c54c-498f-a3d4-b201c2285be5,b0b6a756-8b8a-4cee-9f4f-da0daab9790d,"{'depths': [2704.03369140625, 1004.91400146484...","{'x': [0.00033, 0.00033, 0.000319, 0.000319, 0...","{'channels': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,...",192.993812,493.059209,505.325876,806.408238,3081.895068,1149.644457,3102.680914,3402.730013,3414.996680,3716.062143,4073.404452,4059.372622,1184
6,4b398979-c54c-498f-a3d4-b201c2285be5,a4d69060-1834-4c71-87fe-24eec1988bce,"{'depths': [795.2343139648438, 3011.7927246093...","{'x': [-0.002446, -0.002446, -0.002444, -0.002...",{'uuids': ['f18bcf9c-742a-4366-b75f-e8aa353751...,192.993812,493.059209,505.325876,806.408238,3081.895068,1149.644457,3102.680914,3402.730013,3414.996680,3716.062143,4073.404452,4059.372622,1184
7,d77dd225-f7a9-4d01-996b-82c3ae73d56c,7da0f609-e432-4773-9d6a-40e06a3b206a,"{'amps': [6.607444696949328e-05, 5.99503707103...","{'x': [-0.003783, -0.003783, -0.003794, -0.003...",{'uuids': ['db2f11a6-2190-4014-8243-e11a979d87...,52.456301,352.488467,364.755134,665.820531,3029.724119,1002.943729,3050.437273,3350.486471,3362.753138,3663.834733,4042.941732,3999.650062,1047
8,7149e0fc-a52d-4e93-849c-edc22d54e7a5,0d8aef21-2ac3-4b30-9b9c-0e1627136b9e,"{'clusters': [421, 421, 836, 421, 943, 1482, 8...","{'x': [-9.1e-05, -9.1e-05, -0.0001, -0.0001, -...","{'channels': [0, 13, 0, 0, 0, 2, 1, 1, 1, 1, 1...",52.628383,352.693514,364.960181,666.025344,2894.814311,1009.286428,2915.548829,3215.597395,3227.864061,3528.929458,3941.145379,3872.214240,1065
9,7149e0fc-a52d-4e93-849c-edc22d54e7a5,61d2d753-9f7c-4b10-8d57-29d8735c23e9,"{'amps': [0.0004301588174793437, 0.00018689648...","{'x': [-0.003306, -0.003306, -0.003301, -0.003...","{'depths': [20.0, 20.0, 140.0, 20.0, 140.0, 10...",52.628383,352.693514,364.960181,666.025344,2894.814311,1009.286428,2915.548829,3215.597395,3227.864061,3528.929458,3941.145379,3872.214240,1065


### Just a different way to query the database

In [10]:
eids = one.search(project='psychedelics', dataset=['spikes', 'Camera'], query_type='local')
print(f'Psychedelics sessions with spike sorting & video: {len(eids)}')

Psychedelics sessions with spike sorting & video: 28
